In [ ]:
# importing packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# specifying a file (must be in the working directory)
file ='python_world.xlsx'

# reading the file into Python through pandas
table = pd.read_excel(file)



#copy of table dataframe
full_data = pd.DataFrame.copy(table)

print((full_data[:].isnull().sum() / full_data[:].count()).round(4)*100)

#set a list for countries

countries = full_data['Country Name']

##############

clean_data = full_data.iloc[:,4:]

### loop for missing values

for col in clean_data:
    if clean_data[col].isnull().astype(int).sum()/clean_data[col].count().round(2)>0.3:
        del clean_data[col]
    elif clean_data[col].isnull().astype(int).sum()/clean_data[col].count().round(2)<0.4:        
        a=clean_data[col].median()
        clean_data[col] = clean_data[col].fillna(a) 
    else:
        continue

        

    
#meaningful convertion to ratio 

female_ratio = (clean_data['Population, female'] / clean_data['Population, total'])*100
clean_data['Female Population %'] = female_ratio

male_ratio = (clean_data['Population, male'] / clean_data['Population, total'])*100
clean_data['Male Population %'] = male_ratio

world_gdp = clean_data.loc[14]['GDP (current US$)']
countries_gdp = clean_data[:]['GDP (current US$)']
#world_gdp = int(world_gdp)
gdp_perc = (countries_gdp/world_gdp)*100
clean_data['GDP % on world'] = gdp_perc

# compared flagged values with online sources and replaced them where needed
clean_data.loc[9,'Agriculture, forestry, and fishing, value added (% of GDP)'] = 0
clean_data.loc[3,'Rural population (% of total population)'] = 0
clean_data.loc[9,'Rural population (% of total population)'] = 0
clean_data.loc[12,'Rural population (% of total population)'] = 0

#deep cleaning of non-ratio columns 
del clean_data['Population, female']
del clean_data['Population, male']
del clean_data['Population, total']
del clean_data['GDP (current US$)']
del clean_data['Surface area (sq. km)']
del clean_data['Life expectancy at birth, total (years)']
del clean_data['Mobile cellular subscriptions (per 100 people)']
del clean_data['Population density (people per sq. km of land area)']

clean_data.rename(columns={'Employment in agriculture (% of total employment) (modeled ILO estimate)':'Employment in agriculture (% of total employment) ',
                          'Employment in industry (% of total employment) (modeled ILO estimate)':'Employment in industry (% of total employment)',
                          'Employment in services (% of total employment) (modeled ILO estimate)':'Employment in services (% of total employment)'}, 
                 inplace=True)

header_list = list(clean_data.columns.values)
    
clean_data.insert(0, 'Country', countries)
clean_data1 = clean_data.drop([14])



#barplots for countries
for a in header_list:
    
    plt.grid()
    percent = clean_data[a]
    countries = clean_data['Country']

    x = countries
    y = percent

    plt.xlabel('Countries')
    plt.xticks(rotation=90)
    plt.ylabel('Percentage (%)')
    plt.title(a)
    plt.grid()
    plt.bar(x, y, color='orange')
    plt.show()
    


#distplots
for r in header_list:
    plt.grid()
    sns.distplot(clean_data[r], rug=False, norm_hist=False, fit_kws={"color":"orange"}, color= 'orange')
    plt.xlabel(r)

    plt.tight_layout()
    plt.show()

#histograms  
for r in header_list:
    plt.grid()
    plt.hist(x    =  r,
             data = clean_data,
             bins = 'fd')
    plt.xlabel(r)
    plt.ylabel('Number of countries')
    plt.show()

#boxplots without world 
for x in header_list:
    plt.grid()
    clean_data1.boxplot(column=x)
    plt.show()

clean_data.to_excel('clean data.xlsx')

#barplot to combine services and agricutlural employment in the same graph
clean_data.plot(x="Country", y=["Employment in services (% of total employment)", "Employment in agriculture (% of total employment) "], kind="bar", color = ("green", "orange"), width = 1)

plt.legend(loc='upper right',bbox_to_anchor=(0.3, -0.3))




#clean_data.describe()
#clean_data1.describe()
#clean_data.info
#clean_data1.info

# generating a correlation matrix
clean_data_corr = clean_data1.corr().round(3)

# sending to Excel
clean_data_corr.to_excel("clean_data_corr_matrix.xlsx")

###correlation for undernourishment 

clean_data_corr.loc['Prevalence of undernourishment (% of population)'].sort_values(ascending = False)


#correlation matrix
fig, ax = plt.subplots(figsize=(50,50))
#ax.tick_params(axis="x", labelsize=30)
#ax.tick_params(axis="y", labelsize=30)
#sns.set(font_scale=2)
sns.heatmap(clean_data_corr,
            cmap = 'coolwarm',
            square = True,
            annot = True,
            linecolor = 'black',
            linewidths = 0.5)


plt.tight_layout()

plt.savefig('heatmap_corr.png')
plt.show()


#### lmplots undernoursihment and electricity

#plt.subplot(2,2,1)
sns.lmplot(x = 'Prevalence of undernourishment (% of population)',
           y = 'Access to electricity (% of population)',
           data = clean_data_corr,
           
           scatter_kws= {"marker": "D", 
                        "s": 30, 'color': 'red'}) 
           
plt.title("Undernourishment vs. Access to electricity")

plt.grid()
plt.tight_layout()
plt.savefig("Undernourishment vs. Access to electricity.png")

#plt.subplot(2,2,2)
sns.lmplot(x = 'Prevalence of undernourishment (% of population)',
           y = 'Access to electricity, rural (% of rural population)',
           data = clean_data_corr,
           
           scatter_kws= {"marker": "D", 
                        "s": 30, 'color': 'orange'})

plt.title("Undernourishment vs. Access to electricity (rural)")

plt.grid()
plt.tight_layout()

plt.savefig("Undernourishment vs. Access to electricity (rural).png")


#plt.subplot(2,2,3)
sns.lmplot(x = 'Prevalence of undernourishment (% of population)',
           y = 'Access to electricity, urban (% of urban population)',
           data = clean_data_corr,
           
           scatter_kws= {"marker": "D", 
                        "s": 30, 'color': 'grey'}) 

plt.title("Undernourishment vs. Access to electricity (urban)")

plt.grid()
plt.tight_layout()
plt.show()

plt.savefig("Undernourishment vs. Access to electricity (urban).png")


#### lmplots birth rates and pop

sns.lmplot(x = 'Birth rate, crude (per 1,000 people)',
           y = 'Urban population (% of total population)',
           data = clean_data_corr,
           scatter_kws= {"marker": "D", 
                        "s": 30, 'color': 'red'})

plt.title("Birth rate vs. Urban population")

plt.grid()
plt.tight_layout()
plt.savefig("Birth rate vs. Urban population.png")

sns.lmplot(x = 'Birth rate, crude (per 1,000 people)',
           y = 'Urban population growth (annual %)',
           data = clean_data_corr,
           scatter_kws= {"marker": "D", 
                        "s": 30, 'color': 'red'})

plt.title("Birth rate vs. Urban population growth")

plt.grid()
plt.tight_layout()

plt.savefig("Birth rate vs. Urban population growth.png")

#### lmplots fertility rate and urban population (growth and percentage)

sns.lmplot(x = 'Fertility rate, total (births per woman)',
           y = 'Urban population (% of total population)',
           data = clean_data_corr,
           
           scatter_kws= {"marker": "D", 
                        "s": 30, 'color': 'red'})

plt.title("Fertility rate vs. Urban population")

plt.grid()
plt.tight_layout()
plt.savefig("Fertility rate vs. Urban population.png")


sns.lmplot(x = 'Fertility rate, total (births per woman)',
           y = 'Urban population growth (annual %)',
           data = clean_data_corr,
           
           scatter_kws= {"marker": "D", 
                        "s": 30, 'color': 'red'})

plt.title("Fertility rate vs. Urban population growth")

plt.grid()
plt.tight_layout()

plt.savefig("Fertility rate vs. Urban population growth.png")


#### lmplots fertility rate and rural population 
sns.lmplot(x = 'Fertility rate, total (births per woman)',
           y = 'Rural population (% of total population)',
           data = clean_data_corr,
           
           scatter_kws= {"marker": "D", 
                        "s": 30, 'color': 'red'})

plt.title("Fertility rate vs. Rural population")

plt.grid()
plt.tight_layout()



plt.savefig("Fertility rate vs. Rural population.png")

In [ ]:
print(clean_data1)